# Retrogate Analysis

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-symmetry.ipynb

import pymongo
localMongo = pymongo.MongoClient("mongodb://localhost:27017/")
retroDb = localMongo["retro-database"]
retroCol = retroDb["retro-collection"]

In [ ]:
startBoard = ((' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))
boards = []

def generateBoards(board=startBoard):
    global boards
    white = countStones(('', board), 'w')
    black = countStones(('', board), 'b')
    if (white == 3 and black == 3):
        boards += tuple([board])
        return
    for i in range(0,3):
        for j in range(0,8):
            if board[i][j] == ' ':
                if white == 3:
                    newBoard = place(board, (i, j), 'b')
                    generateBoards(newBoard)
                elif black == 3:
                    newBoard = place(board, (i, j), 'w')
                    generateBoards(newBoard)
                else:
                    for turn in ('w', 'b'):
                        newBoard = place(board, (i, j), turn)
                        generateBoards(newBoard)

def generateBoardNewGen(board=startBoard):
    global boards
    for a1 in range(0,3):
        for at2 in range(7,10):
            a2 = at2 % 8
            for b1 in range(0,3):
                for bt2 in range(7,10):
                    b2 = bt2 % 8
                    for c1 in range(0,3):
                        for ct2 in range(7,12):
                            c2 = ct2 % 8
                            for d1 in range(0,3):
                                for dt2 in range(7,12):
                                    d2 = dt2 % 8
                                    for e1 in range(0,3):
                                        for e2 in range(0,8):
                                            for f1 in range(0,3):
                                                for f2 in range(0,8):
                                                    board1 = place(board, (a1, a2), 'w')
                                                    board2 = place(board1, (b1, b2), 'b')
                                                    board3 = place(board2, (c1, c2), 'w')
                                                    board4 = place(board3, (d1, d2), 'b')
                                                    board5 = place(board4, (e1, e2), 'w')
                                                    board6 = place(board5, (f1, f2), 'b')
                                                    white = countStones(('', board6), 'w')
                                                    black = countStones(('', board6), 'b')
                                                    if (white != 3 or black != 3):
                                                        break;
                                                    boards += tuple([board6])

In [ ]:
def getSymmetryUnique():
    global boards
    doubleBoards = set()
    uniqueStates = []
    stones = tuple([0, 0])
    for board in boards:
        state = tuple([stones, board])
        if not (state in doubleBoards):
            doubleBoards |= findSymmetries(state)
            uniqueStates.append(state)
    print(len(doubleBoards))
    return uniqueStates

In [ ]:
def fillDb(states):
    fullStep = []
    stepCount = 0
    
    stepCount += 1
    _states = states
    for state in _states:
        _nextStates = nextStates(state, 'w')
        for ns in _nextStates:
            if (finished(ns, 'w') and utility(ns, 'w') == 1):
                entry = { "state": state, "nextState": ns, "steps": stepCount }
                retroCol.insert_one(entry)
                fullStep.append(state)
                states.remove(state)
                break
    
    while len(states) > 0:
        print(len(fullStep))
        enrichedFullStep = set()
        for s in fullStep:
            enrichedFullStep |= findSymmetries(s)
        
        _states = states
        halfStep = []
        stepCount += 1
        for state in _states:
            _nextStates = nextStates(state, 'b')
            if (all(ns in enrichedFullStep for ns in _nextStates)):
                halfStep.append(state)
                states.remove(state)
                break
        if len(halfStep) == 0:
            break
        
        print(len(halfStep))
        enrichedHalfStep = set()
        for s in halfStep:
            enrichedHalfStep |= findSymmetries(s)
        
        fullStep = []
        stepCount += 1
        for state in _states:
            _nextStates = nextStates(state, 'w')
            for ns in _nextStates:
                if (ns in enrichedHalfStep):
                    entry = { "state": state, "nextState": ns, "steps": stepCount }
                    retroCol.insert_one(entry)
                    fullStep.append(state)
                    states.remove(state)
                    break
        if len(fullStep) == 0:
            break

In [ ]:
generateBoardNewGen()
print(len(boards))
print(boards[1])
states = getSymmetryUnique()
print(len(states))
fillDb(states)